In [1]:
import time
import requests
import numpy as np
import pandas as pd
from helper import get_authorization_url, get_access_token, get_playlist_urls, get_song_urls, get_spotify_songs_metadata

In [2]:
auth_url = get_authorization_url()
print(auth_url)

https://accounts.spotify.com/authorize?client_id=962aeb5559434e8a8ddb2b8c78e6c3d2&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000&scope=user-read-private+user-read-email


In [3]:
cookie = "AQBCH5fmjllyBj8HnWvuy0yYFEpdWPpcd5m23N4SUa_OKK1v-B-vob8JKTVEPXAZtyaIi5TgGCKPhnFovkDwN3Zbzik5W1xbb0fdq8Rc86eDOW9nZ5r-7jNEPVmu0cdTqYrfusMUJaIMzbpPTKx7RuCrOSDz17U_0QK0wYGrrYVt47yCXcXHStkr2BQQwYQFB-VejFZR1ixSCltE8g"

In [4]:
access_token = get_access_token(cookie)

In [5]:
url = "https://api.spotify.com/v1/me/playlists"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    playlists = response.json()['items']
else:
    print(f'Error: {response.status_code} - {response.json()}')

In [6]:
favorite_songs = []

for playlist in playlists:
  song_url = f"https://api.spotify.com/v1/playlists/{playlist['id']}/tracks"
  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }
  response = requests.get(song_url, headers=headers)

  if response.status_code == 200:
    favorite_songs.extend([song['track']['id'] for song in response.json()['items']])

batched_favorite_songs = [favorite_songs[i:i + 50] for i in range(0, len(favorite_songs), 50)]

In [7]:
# liked_songs = pd.DataFrame()
# for batch in batched_favorite_songs:
#     liked_songs = get_spotify_songs_metadata(liked_songs, access_token, batch, 1, 1)
# liked_songs.to_parquet('./datasets/cleaned/music-recommendation-system/liked/songs.parquet', engine='pyarrow')

In [8]:
genres = ['pop', 'mood', 'gaming']

playlists_based_on_genres = get_playlist_urls(access_token, genres)
unheard_song_ids = get_song_urls(access_token, playlists_based_on_genres)

In [10]:
batches = (len(unheard_song_ids) / 10) + 1
unheard_song_ids_batches = np.array_split(unheard_song_ids, batches)

In [11]:
unheard_songs = pd.DataFrame()

for batch in unheard_song_ids_batches:
    unheard_songs = get_spotify_songs_metadata(unheard_songs, access_token, batch, 0, np.nan)

In [12]:
len(unheard_songs)

1044

In [13]:
unheard_songs.isnull().sum()

id                     0
title                  0
artist(s)              0
popularity             0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
type                   0
uri                    0
track_href             0
analysis_url           0
duration_ms            0
time_signature         0
listen                 0
like                1044
dtype: int64

In [ ]:
unheard_songs.to_parquet('./datasets/cleaned/music-recommendation-system/unheard/songs.parquet', engine='pyarrow')

In [ ]:
discover_weekly_song_ids = []
discover_weekly_url = "https://api.spotify.com/v1/playlists/37i9dQZEVXcHBp4tw3uD8b/tracks"

headers = {
  'Authorization': f'Bearer {access_token}',
  'Content-Type': 'application/json'
}

response = requests.get(discover_weekly_url, headers=headers)

if response.status_code == 200:
  discover_weekly_song_ids = [song['track']['id'] for song in response.json()['items']]
  time.sleep(0.5)
else:
  print(f'Error: {response.status_code} - {response.json()}')

print(discover_weekly_song_ids)

['1s7oOCT8vauUh01PbJD6ps', '1elhmWW7Bv0MOQj2gAsyoV', '1OyakDSOcvuZjvESJxri3w', '3TFdXrYIubXABQh0CxIiSs', '6TkyRlfE4XMPDR6uA0xYyz', '5cRalo5aaLDFcNudQEQqJx', '3kyVmdoNCvLxGlmqY1yTQz', '69g0XAms2Js45eTCrT1nVL', '16C6yAxcA3GCtoXbq3dUYl', '1lF7uD9Yu2CL17a6tm4Tkh', '67u4rpzdgdiPjRGZii9GEz', '73tnLQtl7FqD0MS7VEM0xi', '3nzK0WPzGPczWa45eIFphc', '1OBK4DltPHWoeilauCyckU', '6YULY6STClTO2tekAZb81j', '5tgLkUlAmTR9FhOzVVfIJS', '7B5EbapjMpLuK7A7PJUvxn', '6UdU7dLYydmt6OaPItTso9', '6tm4RYbhLGrFmYwSTFNFaB', '7s0pdQmbXq6GtMHpkCTLLw', '23xnJ8uEgUPXLARnsedrOn', '2VbO6uAFOFVaAuWGbntoQL', '6vhrwbHQjl4klPXlPLnar4', '4UoeRYvYxVx72vWK4lGRYv', '3XFFxrFN9s8hoIO5TwKxIT', '2F8Sx9EPW0bcqTMZDwxEqX', '3cTsh9sZdHHV6Syjcwg8VW', '4Gzpzsew2wogr4iQSlT54W', '3XCoEJvCYqTbM8bhples8N', '3vN4noEugIxWThJUeEJqQJ']


In [ ]:
discover_weekly_songs = pd.DataFrame()

for song_id in discover_weekly_song_ids:
  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }

  track_response = requests.get(
    f"https://api.spotify.com/v1/tracks/{song_id}",
    headers=headers
  ).json()
  
  audio_analysis_response = requests.get(
    f"https://api.spotify.com/v1/audio-features/{song_id}", 
    headers=headers
  ).json()

  time.sleep(0.1)
  row = {
    'id': track_response.get('id'),
    'title': track_response.get('name'),
    'artist(s)': ', '.join([artist['name'] for artist in track_response.get('album').get('artists')]),
    'popularity': track_response.get('popularity'),
    'danceability': audio_analysis_response.get('danceability'),
    'energy': audio_analysis_response.get('energy'),
    'key': audio_analysis_response.get('key'),
    'loudness': audio_analysis_response.get('loudness'),
    'mode': audio_analysis_response.get('mode'),
    'speechiness': audio_analysis_response.get('speechiness'),
    'acousticness': audio_analysis_response.get('acousticness'),
    'instrumentalness': audio_analysis_response.get('instrumentalness'),
    'liveness': audio_analysis_response.get('liveness'),
    'valence': audio_analysis_response.get('valence'),
    'tempo': audio_analysis_response.get('tempo'),
    'type': audio_analysis_response.get('type'),
    'id': audio_analysis_response.get('id'),
    'uri': audio_analysis_response.get('uri'),
    'track_href': audio_analysis_response.get('track_href'),
    'analysis_url': audio_analysis_response.get('analysis_url'),
    'duration_ms': audio_analysis_response.get('duration_ms'),
    'time_signature': audio_analysis_response.get('time_signature'),
    'listen': 0,
    'like': np.nan
  }
  discover_weekly_songs = pd.concat([discover_weekly_songs, pd.DataFrame(row, index=[len(discover_weekly_songs)])])

print(len(discover_weekly_songs))

30


In [ ]:
discover_weekly_heard_list = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
discover_weekly_like_list = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
discover_weekly_songs['listen'] = discover_weekly_heard_list
discover_weekly_songs['like'] = discover_weekly_like_list

In [ ]:
discover_weekly_songs.to_parquet('./datasets/cleaned/music-recommendation-system/unheard/discover-weekly.parquet', engine='pyarrow')